In [2]:
from PIL import Image
import pandas as pd
import os

In [8]:

class MakeLocateDataFrame():
    
    def __init__(self, type: bool):

        self.type = type
        self.root = r'/opt/ml/data/'
        if self.type:
            self.root += 'train'
        else:
            self.root += 'eval'

    def get_csv(self):
        datas = None

        if self.type:
            datas = pd.read_csv(self.root+'/train.csv')
            datas = datas.drop(['race','id'],axis=1)

        else:
            datas = pd.read_csv(self.root+'/info.csv')

        return datas

    def make_image_locate_frame(self, datas):
        
        labeled_dict = None
        #train = True 인 경우
        if self.type:
            # 이미지 경로를 담은 리스트 생성
            image_locate = [self.root+'/images/'+i for i in datas['path']]
            images = []
            # 경로 안에 있는 숨김 파일은 제거하고 이미지만 저장
            for i in image_locate:
                temp = []
                for j in os.listdir(i):
                    if not j[0] == '.':
                        temp.append(i+'/'+j)
                images.append(temp)
            # 파일 이름으로 클래스를 구분하기 위한 문자
            file_name_string = 'min'
            #새롭게 반환할 데이터 프레임의 틀
            labeled_dict = {"label": [],
                            "gender": [],
                            "age": [],
                            "locate": []}

            for i in images:
                for j in i:
                    #/opt/ml/data/train/images/000002_female_Asian_52/normal.jpg 자른 것
                    get_locate_file = j.split('/')
                    #가장 끝의 파일은 이미지 이름이다.
                    label = file_name_string.index(get_locate_file[-1][0])
                    start = label * 6
                    #000002_female_Asian_52 잘라서 굳이 위에 선언한 데이터 불러오지 않고 수행
                    _, sex, _, age = get_locate_file[-2].split('_')
                    age = int(age)
                    if sex == 'female':
                        start +=3
                    if age < 30:
                        pass
                    elif age < 60:
                        start +=1
                    else:
                        start +=2
                    labeled_dict['label'].append(start)
                    labeled_dict['gender'].append(sex)
                    labeled_dict['age'].append(age)
                    labeled_dict['locate'].append(j)

            labeled_dict = pd.DataFrame(labeled_dict, columns=labeled_dict.keys())

        else:
            pass

        return labeled_dict

In [9]:
a = MakeLocateDataFrame(False)

data = a.get_csv()
# a.make_image_locate_frame(data)
pd.read_csv(a.root+'/info.csv')

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,0
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,0
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,0
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0
